In [4]:
from sdRDM import DataModel
from sdrdm_database import DBConnector, create_tables

In [5]:
# Establish a connection to the database
db = DBConnector(
    username="root",
    password="root",
    host="localhost",
    db_name="db",
    port=3306,
    dbtype="mysql",
)

In [6]:
# Fetch the sdRDM schema
lib = DataModel.from_markdown("./model.md")
lib.Root.meta_tree()

Root
├── id
├── value
├── nested
│   └── 0
│       ├── id
│       ├── another_value
│       └── value
└── some_values


ClassNode(/Root, class_name=None, constants={}, id=None, module=None, outer_type=None)

In [7]:
# Create tables for the sdRDM schema
create_tables(
    db_connector=db,
    model=lib.Root,
    markdown_path="./model.md",
)


🚀 Creating tables for data model Root
│
├── Table __model_meta__ not existing. Adding to DB!
├── Added table model 'Root' to __model_meta__ table
├── Model 'Root' already registered. Skipping.
├── Created table 'Root'
├── Created table 'Root_some_values'
├── Added table model 'Root_nested' to __model_meta__ table
├── Created table 'Root_nested'
├── Added foreign key 'Root_id'(Root) to table Root_some_values
├── Added foreign key 'Root_id'(Root) to table Root_nested
│
╰── 🎉 Created all tables for data model Root



In [8]:
# We are now seeking a way to insert an instance of 
# the sdRDM schema into the database.
print(db.connection.table("__model_meta__"))
print(db.connection.table("Root"))
print(db.connection.table("Root_nested"))
print(db.connection.table("Root_some_values"))

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓
┃ table       ┃ specifications                             ┃ github_url ┃ commit_hash ┃ part_of ┃ obj_name ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩
│ !string     │ json                                       │ string     │ string      │ string  │ string   │
├─────────────┼────────────────────────────────────────────┼────────────┼─────────────┼─────────┼──────────┤
│ Root        │ {'objects': [...], 'enums': [...], ... +3} │ NULL       │ NULL        │ NULL    │ Root     │
│ Root_nested │ NULL                                       │ NULL       │ NULL        │ Root    │ Nested   │
└─────────────┴────────────────────────────────────────────┴────────────┴─────────────┴─────────┴──────────┘



┏━━━━━━━━━┳━━━━━━━━━┓
┃ value   ┃ Root_id ┃
┡━━━━━━━━━╇━━━━━━━━━┩
│ float64 │ !string │
└─────────┴─────────┘



┏━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ another_value ┃ value   ┃ Root_nested_id ┃ Root_id ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ string        │ float64 │ !string        │ string  │
└───────────────┴─────────┴────────────────┴─────────┘



┏━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ some_values ┃ Root_id ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━┩
│ !string     │ string  │
└─────────────┴─────────┘



In [9]:
# Create datasets using the API that has been directly generated from the DB
model = db.get_table_api("Root")

dataset1 = model(value=100.0, some_values=["Hello", "World"])
dataset1.add_to_nested(another_value="something")
dataset1.add_to_nested(another_value="something else")

print(dataset1)

Root
├── id = root0
├── value = 100.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested0
│   │       └── another_value = something
│   └── 1
│       └── Nested
│           ├── id = nested1
│           └── another_value = something else
└── some_values = [Hello, World, ...]



In [11]:
# And another one
dataset2 = model(value=20.0, some_values=["Whats", "up"])
dataset2.add_to_nested(another_value="hi there")
dataset2.add_to_nested(another_value="this is dataset 2")

print(dataset2)

Root
├── id = root2
├── value = 20.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = nested4
│   │       └── another_value = hi there
│   └── 1
│       └── Nested
│           ├── id = nested5
│           └── another_value = this is dataset 2
└── some_values = [Whats, up, ...]



In [12]:
# Insert the new dataset into the database
db.insert(dataset1, dataset2, verbose=True)

Added dataset Root (3f9018bb-6d68-4f8c-95ce-50fc8dcb9198)
Added dataset Root (8798dc6f-5129-4aa4-a51a-2c08af7a3209)


In [13]:
# Lets see if it worked
print(db.connection.table("Root"))
print(db.connection.table("Root_nested"))
print(db.connection.table("Root_some_values"))

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ value   ┃ Root_id                              ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ float64 │ !string                              │
├─────────┼──────────────────────────────────────┤
│   100.0 │ 3f9018bb-6d68-4f8c-95ce-50fc8dcb9198 │
│    20.0 │ 8798dc6f-5129-4aa4-a51a-2c08af7a3209 │
└─────────┴──────────────────────────────────────┘



┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ another_value     ┃ value   ┃ Root_nested_id                       ┃ Root_id                              ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string            │ float64 │ !string                              │ string                               │
├───────────────────┼─────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│ something else    │     nan │ 1ac5aa9f-3e7e-47bf-9282-f9f0e6167318 │ 3f9018bb-6d68-4f8c-95ce-50fc8dcb9198 │
│ this is dataset 2 │     nan │ 35d9213c-2d50-468a-ae8e-6a8cc8f560c2 │ 8798dc6f-5129-4aa4-a51a-2c08af7a3209 │
│ something         │     nan │ 9857d37b-fb31-45b5-9774-2de06316365d │ 3f9018bb-6d68-4f8c-95ce-50fc8dcb9198 │
│ hi there          │     nan │ be31a90b-9a14-44f6-a97a-2bcc77373cb9 │ 8798dc6f-5129-4aa4-a51a-2c08af7a3209 │
└─────────

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ some_values ┃ Root_id                              ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ !string     │ string                               │
├─────────────┼──────────────────────────────────────┤
│ Hello       │ 3f9018bb-6d68-4f8c-95ce-50fc8dcb9198 │
│ World       │ 3f9018bb-6d68-4f8c-95ce-50fc8dcb9198 │
│ Whats       │ 8798dc6f-5129-4aa4-a51a-2c08af7a3209 │
│ up          │ 8798dc6f-5129-4aa4-a51a-2c08af7a3209 │
└─────────────┴──────────────────────────────────────┘



In [14]:
# Finally, recover the data from the database by filtering
retrieved = db.get(table="Root", attribute="value", value=100.0,)
print(retrieved[0])

Root
├── id = 3f9018bb-6d68-4f8c-95ce-50fc8dcb9198
├── value = 100.0
├── nested
│   ├── 0
│   │   └── Nested
│   │       ├── id = 1ac5aa9f-3e7e-47bf-9282-f9f0e6167318
│   │       └── another_value = something else
│   └── 1
│       └── Nested
│           ├── id = 9857d37b-fb31-45b5-9774-2de06316365d
│           └── another_value = something
└── some_values = [Hello, World, ...]

